In [1]:
%run ../chap05/dataset.ipynb

In [2]:
import csv
import re
import random
class VideoDataset(Dataset):
    def __init__(self, filenames, timesteps=10):
        super(VideoDataset, self).__init__('video', 'regression')
        self.set_timesteps(timesteps)
        self.temp_red=list()
        self.temp_blue=list()
        self.temp_green=list()
        xs_0=list()
        ys_0=list()
        xs_0_=list()
        ys_0_temp=list()
        xs_0_temp=list()
        self.ext_xs = list()
        xs_0_temp_0=list()
        xs_0_extra=list()
        #print(xs_0)
        list_Total=['A','T','G','C']
        for i in range(10): #무작위로 AsCpf1 정답값 생성
            list_target=random.randint(10,35) #10~35 사이 정수 값 생성
            ys_0.append(list_target)
            
        for i in range(10):  #무작위로 10개 데이터셋 생성
            list_ran=[]
            for j in range(34):  #34개 A,T,G,C 무작위로 뽑기
                list_ran.append(random.choice(list_Total))
            xs_0.append(list_ran)   #최종결과는 2차원 리스트, 차원 크기는 [10, 34]   10은 데이터 개수, 34는 34열 시계열 정보
            
        for i in range(10):  # 10개 데이터셋 one hot vecter로 만들기
            list_ran=[]
            for j in range(34):  #34개 A,T,G,C에 대하여 각각 원햇 벡터 형태로 변환하기.
                if j >=4 or j<28: #중간 24bp 만큼 처리
                    if 'A' == xs_0[i][j]:
                        list_ran.append([1,0,0,0])
                    elif 'T' == xs_0[i][j]:
                        list_ran.append([0,1,0,0])
                    elif 'G' == xs_0[i][j]:
                        list_ran.append([0,0,1,0])
                    else:
                        list_ran.append([0,0,0,1])
                    
            xs_0_.append(list_ran)   #최종결과는 2차원 리스트, 차원 크기는 [10, 34, 4]   10은 데이터 개수, 
                                     #34는 34열 시계열 정보 34는 원햇벡터 차원 크기
        
        for i in range(10):  # 10개 데이터셋 chromatin accessibility 무작위 생성
            list_target=random.randint(100,350) #100~350 사이 정수 값 생성
            xs_0_extra.append(list_target)
                    
    
        
        xs_0_=np.array(xs_0_,dtype=np.float32) #넘비 배열로 형 변환
        xs_0_extra=np.array(xs_0_extra,dtype=np.float32)
        self.xs_0_extra=xs_0_extra.reshape((10,1))
        ys_0=np.array(ys_0,dtype=np.float32)
        ys_0=ys_0.reshape((10,1))
        
        print("xs_0_",xs_0_.shape)#xs_0_ (10, 34, 4)
        print("ys_0",ys_0.shape) #ys_0 (10, 1)
        
        self.divide_data(xs_0_, ys_0, 0.7) # 학습 데이터, 테스트 데이터 분할
        
    def set_timesteps(self, timesteps):
        self.timesteps = timesteps
        self.input_shape = [timesteps+1, 100, 200, 3] #이미지 개수
        self.output_shape = [timesteps+1, 2] #출력 2개의 속력
        
    @property
    def train_count(self):
        return len(self.tr_xs)
    
    def __str__(self):
        return '{}({}, {} train_data)'.format(self.name, self.mode, self.train_count)
    
    def get_train_data(self, batch_size, nth): #학습할때 사용될 데이터를 불러오는 함수입니다.
        from_idx = nth * batch_size        #배치사이즈 개수만큼 데이터를 긁어오기 위해 번호를 만듭니다.
        to_idx = (nth + 1) * batch_size
        tr_X = self.tr_xs[self.indices[from_idx:to_idx]]    # self.indices 함수는 1번부터 데이터개수만큼
        tr_Y = self.tr_ys[self.indices[from_idx:to_idx]]    # 넘버링된 값이 무작위로 정렬되어있는 함수입니다.
        return tr_X, tr_Y                                   # 거기에 적힌 번호의 데이터를 가져와서 출력해줍니다.

    def shuffle_train_data(self, size):  #self.indices 함수를 만드는 과정입니다.
        np.random.seed(int(time.time()))
        self.indices = np.arange(size)
        np.random.shuffle(self.indices)

    def get_test_data(self):            #테스트 데이터를 불러오는 함수입니다.
        return self.te_xs, self.te_ys

    def divide_data(self, xs, ys, tr_ratio=0.7):  #학습 데이터 테스트 데이터 분할하는 함수입니다.
        data_count = 10  #총 10개
        tr_cnt = data_count * tr_ratio  #개수를 나눕니다.
        te_cnt = data_count - (tr_cnt)

        tr_from, tr_to = 0, tr_cnt
        te_from, te_to = tr_cnt, data_count
        print("divide_data,data_count,tr_from,tr_to,te_to",data_count,tr_from,tr_to,te_to)
        self.tr_xs = xs[tr_from:tr_to]   #여기서부터 저기까지 학습데이터,
        self.tr_ys = ys[tr_from:tr_to]   #여기서부터 저기까지 학습될 타겟 데이터
        self.te_xs = xs[te_from:te_to]   #여기서부터 저기까지 테스트 데이터
        self.te_ys = ys[te_from:te_to]   #여기서부터 저기까지 테스트 타겟 데이터
        self.ext_training_xs = self.xs_0_extra[tr_from:tr_to]  #여기서부터 저기까지 학습될 chromatin accessbility input
        self.ext_test_xs = self.xs_0_extra[te_from:te_to] #여기서부터 저기까지 테스트 chromatin accessbility input
        #print(self.tr_xs, self.te_xs)
        
        self.input_shape = xs[0].shape
        self.output_shape = ys[0].shape
        

NameError: name 'Dataset' is not defined

In [3]:
def video_shot_forward_postproc(self, output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square[:,:])
    aux = diff

    return loss, aux

def video_shot_backprop_postproc(self, G_loss, aux):
    diff = aux
    shape = diff.shape

    g_loss_square = np.ones(shape) / np_cpu.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

VideoDataset.forward_postproc = video_shot_forward_postproc
VideoDataset.backprop_postproc = video_shot_backprop_postproc